In [2]:
import pandas as pd

In [3]:
tech=pd.read_csv(r"/users/pawarbharath/internsic science labs/tech.csv")

In [4]:
tech=tech.drop("Unnamed: 0",axis=1)

In [5]:
tool=pd.read_csv(r"/users/pawarbharath/internsic science labs/tool.csv")

In [6]:
tool=tool.drop("Unnamed: 0",axis=1)

In [7]:
ability=pd.read_excel(r'/users/pawarbharath/downloads/onet/Abilities.xlsx',usecols = ["Element Name"])



In [8]:
newability = ability.drop_duplicates(subset=["Element Name"])
print(newability.head())

            Element Name
0     Oral Comprehension
2  Written Comprehension
4        Oral Expression
6     Written Expression
8       Fluency of Ideas


In [9]:
newability.shape

(52, 1)

In [10]:
newability["category"]="ability"

/Users/pawarbharath/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
newability=newability.reset_index()

In [12]:
newability=newability.drop("index",axis=1)

In [13]:
newability["target"]=2

In [14]:
newability.rename(columns = {'Element Name':'Example'}, inplace = True) 

In [15]:
newability.shape

(52, 3)

In [16]:
title=pd.read_excel(r'/users/pawarbharath/downloads/onet/Alternate Titles.xlsx',usecols = ["Alternate Title"])




In [17]:
title.rename(columns = {'Alternate Title':'Example'}, inplace = True) 

In [18]:
title["category"]="title"

In [19]:
title["target"]=3

In [21]:
industry=pd.read_excel(r'/users/pawarbharath/desktop/industry.xlsx')




In [22]:
industry["category"]="industry knowledge"

In [23]:
industry["target"]=4

In [24]:
frame= [tech,tool,newability,title,industry]

In [37]:
frame

[                                    Example    category  target
 0               Adobe Systems Adobe Acrobat  technology       0
 1                           AdSense Tracker  technology       0
 2                            Atlassian JIRA  technology       0
 3               Blackbaud The Raiser's Edge  technology       0
 4      ComputerEase Construction Accounting  technology       0
 ...                                     ...         ...     ...
 29007        Distributed control system DCS  technology       0
 29008                                 Linux  technology       0
 29009                       Microsoft Excel  technology       0
 29010                      Microsoft Office  technology       0
 29011                               Palm OS  technology       0
 
 [29012 rows x 3 columns],
                                Example category  target
 0                   10-key calculators     tool       1
 1                    Desktop computers     tool       1
 2                  

In [25]:
result = pd.concat(frame)
result=result.reset_index()
result=result.drop("index",axis=1)

In [38]:
result

,Example,category,target
0,Adobe Systems Adobe Acrobat,technology,0
1,AdSense Tracker,technology,0
2,Atlassian JIRA,technology,0
3,Blackbaud The Raiser's Edge,technology,0
4,ComputerEase Construction Accounting,technology,0
...,...,...,...
123286,start-ups,industry knowledge,4
123287,conversion-optimization,industry knowledge,4
123288,user interface design,industry knowledge,4
123289,consulting,industry knowledge,4


In [26]:
result.to_csv('categories.csv') 

In [27]:
import seaborn as sns
sns.countplot(result.category)

In [28]:
from sklearn.model_selection import train_test_split
training,validation=train_test_split(result,random_state=42,test_size=0.3)
training=training.drop("target",axis=1)
validation=validation.drop("target",axis=1)
validation["category"].value_counts()

title                 15677
tool                  12604
technology             8656
industry knowledge       33
ability                  18
Name: category, dtype: int64

In [29]:
import fasttext
import csv
from gensim.utils import simple_preprocess
dataset=training
ds=validation

In [30]:
# NLP Preprocess
dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))



In [31]:
# Prefixing each row of the category column with '__label__'
dataset.iloc[:, 1] = dataset.iloc[:, 1].apply(lambda x: '__label__' + x)
ds.iloc[:, 1] = ds.iloc[:, 1].apply(lambda x: '__label__' + x)



In [32]:
# Saving the CSV file as a text file to train/test the classifier
dataset[['Example', 'category']].to_csv('train2.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

ds[['Example', 'category']].to_csv('test2.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")


In [33]:
# Training the fastText classifier
model = fasttext.train_supervised('train2.txt', wordNgrams = 2,lr=0.1,bucket=2000)

test=model.test('test1.txt') 
print(test)

(30823, 0.9670700450961944, 0.9670700450961944)


In [34]:
input_text="startup"

for i in input_text:
    out=model.predict(input_text)
print(out)

(('__label__technology',), array([0.64207143]))


In [35]:
test

#30823 samples pecision and recall

(30823, 0.9670700450961944, 0.9670700450961944)

In [36]:
model.predict(ds.iloc[5, 0])

(('__label__title',), array([0.99948579]))

In [ ]:
text = ''
model.predict(text)

In [ ]:
input_text="startup"

for i in input_text:
    out=model.predict(input_text)
print(out)

In [ ]:
num_records, precision_at_k, recall_at_k = model.test("test.txt")

In [ ]:
num_records

In [ ]:
precision_at_k

In [ ]:
recall_at_k 

In [ ]:
f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)

In [ ]:
f1_at_k

In [ ]:
def print_results(model, data, k):


    print("records\t{}".format(num_records))
    print("Precision@{}\t{:.3f}".format(k, precision_at_k))
    print("Recall@{}\t{:.3f}".format(k, recall_at_k))
    print("F1@{}\t{:.3f}".format(k, f1_at_k))
    print()
    

In [ ]:
for k in range(1, 3):

    print('test metrics:')
    print_results(model, "test.txt", k)

In [ ]:
model.save_model("fast_filename.ftz")

In [ ]:
#model.save_model("model_filename.bin")

In [ ]:
import time
start=time.time()
model = fasttext.train_supervised('train2.txt', wordNgrams = 2,lr=0.1,bucket=2000)
#model.predict("machine learning")
stop=time.time()
print(stop-start)

In [ ]:
model.predict('machine learning')

In [ ]:
data1 = pd.read_csv('keywords.txt' ,header=None)

In [ ]:
data1

In [ ]:
data1=data1.rename(columns={0:"keywords"})

In [ ]:
output=[]
for i in data1["keywords"]:
    label=model.predict(i)
    output.append(i)
    print(output)
    